# Inference

## Setup

In [1]:
import cv2
import pandas as pd
import numpy as np

top_dir = '/Users/diego/projects/itesoS3/machine-learning/machine-learning-s3'

### transformations setup ###
# MediaPipe  
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

mediapipe_hand_landmarker_path=top_dir+'/models/hand_landmarker.task'
base_options = python.BaseOptions(model_asset_path=mediapipe_hand_landmarker_path)
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=1)
detector = vision.HandLandmarker.create_from_options(options)

# Load
NUM_CLASSES = 'two-classes'
numbers_to_classes = {0:'a', 1:'b'}
ph2_labels_path = top_dir+f"/data/processed/ph2/{NUM_CLASSES}/ph2_parquet_files"
ph2_data = pd.read_parquet(ph2_labels_path, engine='fastparquet')
ph2_landmark_col_names = []
for i in range(21):
    ph2_landmark_col_names.append("b"+str(i)+"x")
    ph2_landmark_col_names.append("b"+str(i)+"y")
    ph2_landmark_col_names.append("b"+str(i)+"z")
normal_vector_columns = ['v3x', 'v3y', 'v3z']
center_of_gravity_columns = ['mean_x', 'mean_y', 'mean_z']
ph2_X = ph2_data[ph2_landmark_col_names + normal_vector_columns + center_of_gravity_columns]

# Scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(ph2_X)
ph2_X_scaled = scaler.transform(ph2_X.values)

# Cosine Kernel PCA
from sklearn.decomposition import KernelPCA
cos_kern_pca_reducer_10 = KernelPCA(n_components=10, kernel='cosine')
cos_kern_pca_reducer_10.fit(ph2_X_scaled)

### Loading our model ###
import pickle
cos_pca_model_filename = top_dir+f'/models/cos_pca_k{1}_n{10}.sav'

I0000 00:00:1731893062.961016   15905 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1731893062.967249  152537 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731893062.974808  152537 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/anaconda3/envs/itesoS3/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## Transformations

In [2]:
def ph1_transform(image_mp):
    """
    SEE dsc_project_data_prep_ph1_transform FOR EXPLANATIONS
    """
    detection_result = detector.detect(image_mp)
    hand_landmarks = detection_result.hand_landmarks
    if hand_landmarks == []:
        return -1, -1
    else:
        hand_landmarks = hand_landmarks[0]
        handedness = detection_result.handedness[0][0]

        landmarks = [0 for _ in range(21*3)]

        for i in range(21):
            landmarks[3*i] = hand_landmarks[i].x
            landmarks[3*i+1] = hand_landmarks[i].y
            landmarks[3*i+2] = hand_landmarks[i].z

        return landmarks, handedness

In [3]:
def ph2_transform(landmarks, handedness):
    """
    SEE dsc_project_data_prep_ph2_transform FOR EXPLANATIONS
    """
    landmarks_x = [landmarks[3*i] for i in range(21)]
    landmarks_y = [landmarks[3*i+1] for i in range(21)]
    landmarks_z = [landmarks[3*+2] for i in range(21)]

    ### Center of gravity ###
    mean_x = np.array(landmarks_x).mean()
    mean_y = np.array(landmarks_y).mean()
    mean_z = np.array(landmarks_z).mean()

    ### Vector normal to the palm ###
    # Points 0, A and B
    zero_x, zero_y, zero_z = landmarks_x[0], landmarks_y[0], landmarks_z[0]
    A_x, A_y, A_z = (landmarks_x[5] + landmarks_x[9])/2, (landmarks_y[5] + landmarks_y[9])/2, (landmarks_z[5] + landmarks_z[9])/2
    B_x, B_y, B_z = (landmarks_x[13] + landmarks_x[17])/2, (landmarks_y[13] + landmarks_y[17])/2, (landmarks_z[13] + landmarks_z[17])/2
    # Vectors 0A and 0B
    zeroA_x, zeroA_y, zeroA_z = A_x - zero_x, A_y - zero_y, A_z - zero_z
    zeroB_x, zeroB_y, zeroB_z = B_x - zero_x, B_y - zero_y, B_z - zero_z
    # Norms for vectors 0A and 0B
    n0A = np.sqrt(zeroA_x**2 + zeroA_y**2 + zeroA_z**2)
    n0B = np.sqrt(zeroB_x**2 + zeroB_y**2 + zeroB_z**2)
    # Vectors v1 and v2 (normalized 0B and 0A)
    v1_x, v1_y, v1_z = zeroB_x/n0B, zeroB_y/n0B, zeroB_z/n0B
    v2_x, v2_y, v2_z = zeroA_x/n0A, zeroA_y/n0A, zeroA_z/n0A
    # Vector v3 (cross product of v1 and v2, or v2 and v1 (depnding on handedness))
    v3_x = v1_y * v2_z - v1_z * v2_y
    v3_y = v1_z * v2_x - v1_x * v2_z
    v3_z = v1_x * v2_y - v1_y * v2_x
    if handedness.display_name == "Left": # v3 = v1 x v2
        pass
    elif handedness.display_name == "Right": # v3 = v2 x v1
        v3_x = -1 * v3_x
        v3_y = -1 * v3_y
        v3_z = -1 * v3_z
    else:
        print("Problem with handedness")
        return landmarks
    
    ### Moving the origin of the frame of reference ###
    wrist_landmarks_x = [landmarks_x[i] - landmarks_x[0] for i in range(21)]
    wrist_landmarks_y = [landmarks_y[i] - landmarks_y[0] for i in range(21)]
    wrist_landmarks_z = [landmarks_z[i] - landmarks_z[0] for i in range(21)]

    ### Changing base ###
    # Cofactors for the matrix (v1, v2, v3)
    A_11, A_21, A_31 = v2_y * v3_z - v2_z * v3_y, v2_x * v3_z - v2_z * v3_x, v2_x * v3_y - v2_y *v3_x
    A_12, A_22, A_32 = v1_y * v3_z - v1_z * v3_y, v1_x * v3_z - v1_z * v3_x, v1_x * v3_y - v1_y *v3_x
    A_13, A_23, A_33 = v1_y * v2_z - v1_z * v2_y, v1_x * v2_z - v1_z * v2_x, v1_x * v2_y - v1_y *v2_x
    # Determinant for the matrix (v1, v2, v3)
    det = v1_x * A_11 - v2_x * A_12 + v3_x * A_13
    # Inverse of the matrix (v1, v2, v3)
    I_11, I_12, I_13 = A_11 / det, A_21 / det, A_31 / det
    I_21, I_22, I_23 = A_12 / det, A_22 / det, A_32 / det
    I_31, I_32, I_33 = A_13 / det, A_23 / det, A_33 / det
    # Changing base
    new_base_landmarks_x = [I_11 * wrist_landmarks_x[i] + I_12 * wrist_landmarks_y[i] + I_13 * wrist_landmarks_z[i] for i in range(21)]
    new_base_landmarks_y = [I_21 * wrist_landmarks_x[i] + I_22 * wrist_landmarks_y[i] + I_23 * wrist_landmarks_z[i] for i in range(21)]
    new_base_landmarks_z = [I_31 * wrist_landmarks_x[i] + I_32 * wrist_landmarks_y[i] + I_33 * wrist_landmarks_z[i] for i in range(21)]

    ### Putting it all together ###
    transformed_landmarks = []
    for i in range(21):
        transformed_landmarks.append(new_base_landmarks_x[i])
        transformed_landmarks.append(new_base_landmarks_y[i])
        transformed_landmarks.append(new_base_landmarks_z[i])
    transformed_landmarks = transformed_landmarks + [v3_x, v3_y, v3_z] + [mean_x, mean_y, mean_z]

    return transformed_landmarks

In [4]:
def ph3_transform(transformed_landmarks, reducer_name):
    """
    SEE dsc_project_data_prep_ph3_transform FOR EXPLANATIONS
    """
    # Scaling
    scaled_landmarks = scaler.transform(transformed_landmarks)

    # Feature Selection
    if reducer_name=="cos_pca":
        reducer = cos_kern_pca_reducer_10

    selected_features = reducer.transform(scaled_landmarks)
    return selected_features


## Loading the model

In [5]:
# load the model from disk
cos_pca_model = pickle.load(open(cos_pca_model_filename, 'rb'))
cos_pca_model

KNeighborsClassifier(n_neighbors=1)

## Inference using the webcam

In [6]:
def classify_webcam_image(reducer_name):
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Convert the frame to a PIL Image
        image_mp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_mp = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_mp)

        # Extract landmarks
        landmarks, handedness = ph1_transform(image_mp)
        
        if (landmarks, handedness) == (-1, -1):
            cv2.putText(frame, "No hand detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
            cv2.imshow('Webcam', frame)
        else:
            # Perform geometric transformations
            transformed_landmarks = ph2_transform(landmarks, handedness)

            # Select features using UMAP
            transformed_landmarks = np.array(transformed_landmarks).reshape(1, -1)
            selected_features = ph3_transform(transformed_landmarks, reducer_name)

            # Make prediction
            if reducer_name=="cos_pca":
                model = cos_pca_model
            
            pred = model.predict(selected_features)

            # Display the framge with the prediction
            cv2.putText(frame, f'Predicted Class: {numbers_to_classes[pred[0]]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
            cv2.imshow('Webcam', frame)
        
        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [7]:
classify_webcam_image("cos_pca")

2024-11-17 19:24:26.428 python[1886:15905] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
W0000 00:00:1731893067.714986  152547 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/opt/anaconda3/envs/itesoS3/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/opt/anaconda3/envs/itesoS3/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/itesoS3/lib/pytho